In [ ]:
%matplotlib inline
import numpy as np

import maxwell_2d

# important to plot
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
from celluloid import Camera

# output animation as html5
rc('animation', html='html5')

plt.style.use('dark_background')

# animation function. This is called sequentially
def get_solution(i, state = 0):
    sol = claw.frames[i].states[state].get_q_global()
    return sol

def plot_q(frame, x, y, q = 1, ax = None, title = None):
    data = get_solution(frame)
    if title is not None:
        ax.set_title(title)
    else:
        ax.set_title('frame '+str(frame));
    im = ax.pcolormesh(x, y, data[q], shading = 'auto', animated = True);
    return im

def get_q_max_min():
    data = get_solution(i)
    [data[k].max() for k in len(data)]
    
def pcolor_q(i, ax, x, y, vmin = None, vmax = None, fig = None, colorbar = False):
    data = get_solution(i)
    for k in range(3):
        im = ax[k].pcolormesh(x, y, data[k], shading = 'auto', vmin = vmin[k], vmax = vmax[k])
        if colorbar:
            fig.colorbar(im, ax = ax[k])
        ax[k].grid(True)
        ax[k].set_title('q = ' + str(k) + ' @ frame = ' + str(i))
        ax[k].set_ylabel('y')
        ax[k].set_xlabel('x')
    return

def myanimate(i, ax, x, y, vmin = None, vmax = None):
    _ = pcolor_q(i, ax, x, y, vmin = vmin, vmax = vmax)


In [ ]:
# maxwell_2d.x_upper = 200
claw = maxwell_2d.em2D(mx = 256, num_frames = 20, heading = 'x', shape = 'pulse')
claw.keep_copy = True
claw.tfinal = 100
# claw.logger.setLevel(logging.CRITICAL)
claw.run()

In [ ]:
dmax = []
dmin = []
dmin_th = 1e-12
for i in range(len(claw.frames)):
    data = get_solution(i)
#     dini = np.zeros(3)
    dmax.append(list(np.max(data, axis = 1).max(axis=1)))
    dmin.append(list(np.min(data, axis = 2).min(axis=1)))
dmax = np.max(dmax, axis = 0)
dmin = np.min(dmin, axis = 0)
dmin[dmin<dmin_th]=0

In [ ]:
dmax

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(nrows = 3, sharex = True, figsize = (10, 10))
xc, yc = claw.solution.domain.grid.p_centers
pcolor_q(20, ax, xc, yc, fig = fig, colorbar = True, vmin = dmin, vmax = dmax)

In [ ]:
%%capture out
fig, ax = plt.subplots(nrows = 3, sharex = True, figsize = (10, 10))
ani = animation.FuncAnimation(fig, myanimate, frames = len(claw.frames), fargs = (ax, xc, yc, dmin, dmax), interval = 1e3)

In [ ]:
HTML(ani.to_html5_video())

In [ ]:
# %%capture out
# fig, ax = plt.subplots(figsize = (10,10));

# ax.set_xlim((maxwell_2d.x_lower, maxwell_2d.x_upper))
# ax.set_ylim((maxwell_2d.y_lower, maxwell_2d.y_upper))

# ims = []
# for i in range(10):
#     ims.append([plot_q(i, xc, yc, ax = ax)]);

In [ ]:
# animation.ArtistAnimation(fig, ims, interval=1000, blit=True, repeat_delay=1000)